In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202540.csv")

In [3]:
ema = ema.copy()

# -------------------------------
# 1. Calcular venta_costo_ema
# -------------------------------
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# -------------------------------
# 2. Totales por sucursal
# -------------------------------
ema["venta_costo_tot"] = ema.groupby("store_name")["venta_costo_ema"].transform("sum")
ema["total_ema"] = ema.groupby("store_name")["EMA"].transform("sum")

# -------------------------------
# 3. Proporciones dentro de cada sucursal
# -------------------------------
ema["venta_costo%"] = (ema["venta_costo_ema"] / ema["venta_costo_tot"]).round(4)
ema["cantidad%"] = (ema["EMA"] / ema["total_ema"]).round(4)

# -------------------------------
# 4. Orden y acumulados
# -------------------------------
# Ordenar por costo dentro de cada sucursal
ema = ema.sort_values(["store_name", "venta_costo_ema"], ascending=[True, False])
ema["acumulado_costo"] = ema.groupby("store_name")["venta_costo%"].cumsum().round(4)

# Ordenar por cantidad dentro de cada sucursal
ema = ema.sort_values(["store_name", "EMA"], ascending=[True, False])
ema["acumulado_cantidad"] = ema.groupby("store_name")["cantidad%"].cumsum().round(2)

# Acumulado de desviación por sucursal
if "desviacion_ema%" in ema.columns:
    ema["acumulado_desviacion"] = (
        ema.groupby("store_name")["desviacion_ema%"].cumsum().round(2)
    )

# -------------------------------
# 5. Renombrar columnas (estilo reporte Odoo)
# -------------------------------
ema_def = ema.rename(
    columns={
        "año": "Año",
        "semana_num": "semana",
        "EMA": "Ema",
        "producto_costo_unitario": "Costo unitario",
        "venta_costo_ema": "Venta costo EMA",
        "venta_costo%": "Venta costo %",
        "acumulado_costo": "Acumulado costo",
        "desviacion_ema%": "Desviacion EMA %",
    }
)

# -------------------------------
# 6. Calcular desviación en dinero
# -------------------------------
if "Desviacion EMA %" in ema_def.columns:
    ema_def["Desviacion EMA"] = (
        ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]
    ).round(1)



# Ejemplo de reglas (lo que viene de conf_acumulado_desviacion en Odoo) #PERILLA
reglas = [
    {"rango_ini": 0, "rango_fin": 0.8, "valor": 0},
    {"rango_ini": 0.8, "rango_fin": 1.5, "valor": 0.5},
    {"rango_ini": 1.5, "rango_fin": 3, "valor": 0.8},
    {"rango_ini": 3, "rango_fin": 2000, "valor": 1},
]

# aplicar reglas 0–0.8–1 como antes, pero sobre coef_ventas_norm


# Función que asigna el valor según reglas
def asignar_desviacion(coef, reglas):
    for r in reglas:
        if r["rango_ini"] <= coef <= r["rango_fin"]:
            return r["valor"]
    return np.nan  # si no entra en ningún rango

# Aplicar
ema_def["acumulado_desviacion_p"] = ema_def["coef_ventas"].apply(
    lambda x: asignar_desviacion(x, reglas)
)



# -------------------------------
# 7. Limpiar columnas innecesarias
# -------------------------------
if "Unnamed: 0" in ema_def.columns:
    ema_def = ema_def.drop(columns=["Unnamed: 0"])


# 8. Reglas acumulado costo combinado
# --- Configuración ---
def check_div(a, b):
    try:
        return a / b if b != 0 else 0
    except Exception:
        return 0

## PERILLA

conf_porcentajes_impacto = {
    "cantidad": 0.5,
    "costo": 0.3,
    "desviacion": 0.2,
    "cantidad_mayor": 0.7,
    "costo_mayor": 0.3,
}

# Valor de referencia en porcentaje (ejemplo: 10%) ##PERILLA
conf_acumulado_cantidad = 80
conf_acumulado_cantidad_desviacion = check_div(conf_acumulado_cantidad, 100)


# --- Cálculo en el DataFrame ---
def calcular_combinado(row):
    acumulado_cantidad_final = round(row["acumulado_cantidad"], 2)
    acumulado_costo_final = round(row["Acumulado costo"], 2)
    acumulado_desviacion_p_final = round(row["acumulado_desviacion_p"], 2)

    if acumulado_cantidad_final >= conf_acumulado_cantidad_desviacion:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad_mayor"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo_mayor"]
        )
    else:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo"]
            + acumulado_desviacion_p_final * conf_porcentajes_impacto["desviacion"]
        )

# Nueva columna en tu df
ema_def["acumulado_combinado"] = ema_def.apply(calcular_combinado, axis=1).round(2)

ema_def["acumulado_combinado"] = (
    ema_def.groupby("store_name")["acumulado_combinado"]
    .transform(lambda x: x / x.max())
).round(2)


def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.8:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

ema_def["Clasificacion"] = ema_def["acumulado_combinado"].apply(clasificar_pareto)


In [4]:
ema_ord = ema_def[["store_name","product_ref","Año","semana","Ema",
"Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo","Desviacion EMA",
"Desviacion EMA %","acumulado_cantidad","coef_ventas","acumulado_desviacion","acumulado_desviacion_p","acumulado_combinado","Clasificacion"]]

ema_ord = ema_ord[~(ema_ord["store_name"]=="0")]

In [5]:
ema_ord["acumulado_combinado"].describe()

count    432000.000000
mean          0.978292
std           0.089147
min           0.010000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: acumulado_combinado, dtype: float64

In [6]:
ema_ord[(ema_ord["store_name"]=="SUCURSAL NORTE") & (ema_ord["product_ref"]=="DAE02286025")]

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion
284392,SUCURSAL NORTE,DAE02286025,2025.0,38,1.648922,97279.53,160406.36,0.0007,0.2326,1.393389e+10,86866.20,0.52,0.0,4857778.65,0.0,0.37,AAA
284393,SUCURSAL NORTE,DAE02286025,2025.0,39,1.319138,97279.53,128325.12,0.0006,0.2742,1.065098e+05,0.83,0.57,11.0,6123340.33,1.0,0.63,A
284394,SUCURSAL NORTE,DAE02286025,2025.0,40,1.055310,97279.53,102660.06,0.0005,0.3308,1.077931e+05,1.05,0.63,12.0,8266998.71,1.0,0.68,A
284395,SUCURSAL NORTE,DAE02286025,2025.0,41,0.844248,97279.53,82128.05,0.0004,0.3892,9.526850e+04,1.16,0.67,13.0,10805324.94,1.0,0.72,A
284396,SUCURSAL NORTE,DAE02286025,2025.0,42,0.675398,97279.53,65702.40,0.0003,0.4432,8.081400e+04,1.23,0.72,14.0,14044535.36,1.0,0.77,A
284397,SUCURSAL NORTE,DAE02286025,2025.0,43,0.540318,97279.53,52561.88,0.0002,0.5048,6.675360e+04,1.27,0.78,15.0,16825208.57,1.0,0.82,B
284382,SUCURSAL NORTE,DAE02286025,2025.0,28,0.320000,97279.53,31129.45,0.0001,0.6512,0.000000e+00,0.00,0.90,0.0,25620309.08,0.0,0.91,B
284383,SUCURSAL NORTE,DAE02286025,2025.0,29,0.256000,97279.53,24903.56,0.0001,0.7014,0.000000e+00,0.00,0.90,0.0,29846877.12,0.0,0.93,B
284384,SUCURSAL NORTE,DAE02286025,2025.0,30,0.230400,97279.53,22413.20,0.0001,0.7258,1.014306e+09,45254.83,0.90,0.0,32857876.92,0.0,0.94,B
284385,SUCURSAL NORTE,DAE02286025,2025.0,31,0.207360,97279.53,20171.88,0.0001,0.7509,9.309536e+08,46151.06,0.90,0.0,36713169.72,0.0,0.96,C


## NIVEL DE SERVICIO

In [7]:
# Definimos las condiciones para la meta, basadas en la clasificación
condiciones_meta = [
    (ema_ord['Clasificacion'] == 'AAA'),
    (ema_ord['Clasificacion'] == 'A'),
    (ema_ord['Clasificacion'] == 'B'),
    (ema_ord['Clasificacion'] == 'C')
]

# Definimos los valores de la meta que se asignarán a cada clasificación
metas = [0.999, 0.95, 0.90, 0.50]

ema_ord = ema_ord.copy()

# Asignamos la nueva columna 'Meta' usando np.select
ema_ord.loc[:, 'Nivel de servicio'] = np.select(condiciones_meta, metas, default=np.nan)

In [8]:
ema_ord.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv")

bq = ema_ord[ema_ord["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False)

bq.head((5))

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
205638,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,32,63.578197,13771.02,875536.62,0.0009,0.1010,166352.0,0.19,0.00,1.416667,0.19,0.5,0.15,AAA,0.999
205639,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,33,57.062558,13771.02,785809.63,0.0008,0.1104,172878.1,0.22,0.00,3.221831,0.41,1.0,0.26,AAA,0.999
205647,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,41,55.266503,13771.02,761076.12,0.0008,0.1144,152215.2,0.20,0.01,4.166375,0.61,1.0,0.28,AAA,0.999
205640,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,34,54.450046,13771.02,749832.67,0.0008,0.1168,179959.8,0.24,0.01,3.485714,0.85,1.0,0.28,AAA,0.999
247274,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,41,51.413950,10117.70,520190.92,0.0005,0.1987,124845.8,0.24,0.01,4.302817,1.09,1.0,0.30,AAA,0.999


In [9]:
df_mio = ema_ord[["store_name","product_ref","semana","Ema","acumulado_combinado","Clasificacion"]]


In [10]:
conteo = pd.crosstab(df_mio["store_name"], df_mio["Clasificacion"])
conteo

Clasificacion,A,AAA,B,C
store_name,,,,
PRINCIPAL COTA,1504,341,1656,50499
SUCURSAL BARRANQUILLA,1903,659,2099,49339
SUCURSAL BUCARAMANGA,1994,692,2269,49045
SUCURSAL CALI,1784,523,2163,49530
SUCURSAL CALLE 6,1796,751,2213,49240
SUCURSAL MEDELLIN,1542,524,1872,50062
SUCURSAL NORTE,1607,485,2010,49898
SUCURSAL VALLADOLID,1425,443,1926,50206


In [11]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

productos_ema = models.execute_kw(
    db, uid, password,
    "cs.analisis.costo", "search_read",[],
    #[
    #    [["semana","=",36]]
    #],
    {"fields": ["sucursal_id","product_name","ano","semana","ema","producto_costo_unitario","acumulado_combinado","clasificacion","almacen_id" ]}
)

df = pd.DataFrame(productos_ema)

def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["store_id_num"] = df["sucursal_id"].apply(extract_id)
df["store_name"]   = df["sucursal_id"].apply(extract_name)

df["almacen_id_num"] = df["almacen_id"].apply(extract_id)
df["almacen_name"]   = df["almacen_id"].apply(extract_name)

df_real =df.copy()


In [12]:
df_mio.head()

,store_name,product_ref,semana,Ema,acumulado_combinado,Clasificacion
401017,PRINCIPAL COTA,DCS10536137,31,31.662640,0.19,AAA
401014,PRINCIPAL COTA,DCS10536137,28,29.493333,0.01,AAA
401018,PRINCIPAL COTA,DCS10536137,32,28.496376,0.20,AAA
401015,PRINCIPAL COTA,DCS10536137,29,26.744000,0.02,AAA
71903,PRINCIPAL COTA,BCE00606125,32,26.686976,0.04,AAA


In [13]:
df_real = df_real[["store_name","product_name","semana","ema","acumulado_combinado","clasificacion"]]

In [14]:
df_real

,store_name,product_name,semana,ema,acumulado_combinado,clasificacion
0,SUCURSAL BARRANQUILLA,DAB02570025,42,44.853995,0.257319,AAA
1,SUCURSAL BARRANQUILLA,DAB14570025,42,43.550653,0.260646,AAA
2,SUCURSAL BARRANQUILLA,BCS00035125,42,29.347685,0.240800,AAA
3,SUCURSAL BARRANQUILLA,BCS00025125,42,28.324394,0.230885,AAA
4,SUCURSAL BARRANQUILLA,DAB02772025,42,27.739197,0.352331,AAA
...,...,...,...,...,...,...
43852,SUCURSAL NORTE,DAE05466003,42,0.000000,1.000000,C
43853,SUCURSAL NORTE,DAE05094003,42,0.000000,1.000000,C
43854,SUCURSAL NORTE,DAE06511003,42,0.000000,1.000000,C
43855,SUCURSAL NORTE,DAE06627002,42,0.000000,1.000000,C
